In [1]:
#Ignore scikit learn deprication warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


In [2]:
import os
print(os.getcwd())
#os.chdir('../blocking/')
import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re
from numpy import genfromtxt

/Users/andrew/workspace/endangeredanimals/analysis


In [3]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import *
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import *

In [4]:
t = pd.read_csv("../learning/test.csv", encoding="ISO-8859-1", index_col='_id')
#Automatically label the testing data
matches = 0;
nonmatches = 0;
labels = []
for index, row in t.iterrows():
    if row['rtable_scientific_name'].strip().lower() == row['ltable_scientific_name'].strip().lower():
        labels.append(1)
        matches = matches + 1
    else:
        labels.append(0)
        nonmatches = nonmatches + 1
        
t['label'] = labels

#cols_to_keep version 2
cols_to_keep = ['name_name_jac_qgm_3_qgm_3',
                'name_name_jac_dlm_dc0_dlm_dc0', 
                'countries_countries_jac_qgm_3_qgm_3',
                'countries_countries_cos_dlm_dc0_dlm_dc0',
                'countries_countries_jac_dlm_dc0_dlm_dc0', 
                'countries_countries_mel',
                'countries_countries_lev_dist', 
                'countries_countries_lev_sim',
                'countries_countries_nmw', 
                'countries_countries_sw',
                'country_count_country_count_exm', 
                'country_count_country_count_anm',
                'country_count_country_count_lev_dist',
                'country_count_country_count_lev_sim', 
                'status_match',
                'country_overlap', 
                'country_count_sim']

tr = t[cols_to_keep + ['label']]
tr.to_csv('../learning/test_reduced.csv')

In [5]:
# Awesome sauce.  Now let's fill in missing values rather than dropping them
# reload train
train = genfromtxt('../learning/train_reduced.csv',delimiter=',')
test = genfromtxt('../learning/test_reduced.csv',delimiter=',')
unlabeled_r = genfromtxt('../learning/unlabeled_reduced.csv',delimiter=',')


# get rid of first row (I think this is just column labels)
train = train[1:,:]
print('train', train.shape)

test = test[1:,:]
print('test', test.shape)


unlabeled_r = unlabeled_r[1:,:]
print('unlabeled', unlabeled_r.shape)



##Drop all nans:
train = train[~np.isnan(train).any(axis=1)]
test = test[~np.isnan(test).any(axis=1)]
unlabeled_r = unlabeled_r[~np.isnan(unlabeled_r).any(axis=1)]

print('test', test.shape)
print(test[6,:])


yTrue = test[:,test.shape[1]-1] # label
print(yTrue.shape)
print(yTrue[6])

Xtest = test[:,1:test.shape[1]-1]  # data
print(Xtest.shape)
print(Xtest[6])

y = train[:,train.shape[1]-1] # label
X = train[:,1:train.shape[1]-1]  # data

unlabeled_X = unlabeled_r[:,1:unlabeled_r.shape[1]]


train (279, 19)
test (120, 19)
unlabeled (2179, 18)
test (59, 19)
[  9.06326700e+06   1.79487179e-01   3.33333333e-01   0.00000000e+00
   0.00000000e+00   0.00000000e+00   3.53131313e-01   2.30000000e+01
   8.00000000e-02  -1.20000000e+01   2.00000000e+00   0.00000000e+00
   5.00000000e-01   1.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   1.00000000e+00   0.00000000e+00]
(59,)
0.0
(59, 17)
[  0.17948718   0.33333333   0.           0.           0.           0.35313131
  23.           0.08       -12.           2.           0.           0.5
   1.           0.           0.           0.           1.        ]


In [6]:
# Double check that model is good
from sklearn.grid_search import GridSearchCV


# SVC tuning
#tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000]},
#                     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
#clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=10,
#                       scoring='f1')
clf = SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf', max_iter=-1, probability=False, random_state=None, shrinking=True, tol=0.001, verbose=False)
clf.fit(X,y)

# Get parameters from model
#params = clf.get_params()
#sv = clf.support_vectors
#nv = clf.n_support_
#a  = clf.dual_coef_
#b  = clf._intercept_
#cs = clf.classes_



yPred = clf.predict(Xtest)
prec, rec, f1, whatever = precision_recall_fscore_support(yTrue, yPred,average='weighted')
print('prec', prec)
print('rec', rec)
print('f1', f1)


prec 0.894292632307
rec 0.898305084746
f1 0.894741910632


In [7]:
#import gold reduced and train the classifier on the full thing
gold = genfromtxt('../learning/gold_reduced.csv',delimiter=',')


# get rid of first row (I think this is just column labels)
gold = gold[1:,:]

##Drop all nans:
gold = gold[~np.isnan(gold).any(axis=1)]

gold_y = gold[:,gold.shape[1]-1] # label
gold_X = gold[:,1:gold.shape[1]-1]  # data


In [9]:
unlabeled = em.read_csv_metadata("../learning/unlabeled.csv", encoding="ISO-8859-1", key='_id')
df = unlabeled.dropna(subset = ['_id', 'name_name_jac_qgm_3_qgm_3', 'name_name_jac_dlm_dc0_dlm_dc0',
       'countries_countries_jac_qgm_3_qgm_3',
       'countries_countries_cos_dlm_dc0_dlm_dc0',
       'countries_countries_jac_dlm_dc0_dlm_dc0', 'countries_countries_mel',
       'countries_countries_lev_dist', 'countries_countries_lev_sim',
       'countries_countries_nmw', 'countries_countries_sw',
       'country_count_country_count_exm', 'country_count_country_count_anm',
       'country_count_country_count_lev_dist',
       'country_count_country_count_lev_sim', 'status_match',
       'country_overlap', 'country_count_sim'])

#labeled.head()
#print(test[2])
#labeled_r.columns

Metadata file is not present in the given path; proceeding to read the csv file.


In [10]:
#train model on gold
clf = SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf', max_iter=-1, probability=False, random_state=None, shrinking=True, tol=0.001, verbose=False)
clf.fit(gold_X,gold_y)

prediction = clf.predict(unlabeled_X)

In [13]:
unlabeled['label'] = prediction
unlabeled
#len(dropped)

,_id,ltable_id,rtable_id,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,...,countries_countries_nmw,countries_countries_sw,country_count_country_count_exm,country_count_country_count_anm,country_count_country_count_lev_dist,country_count_country_count_lev_sim,status_match,country_overlap,country_count_sim,label
0,2103728,6253,265,light red meranti,Shorea,Dipterocarpaceae,Terrestrial,Indonesia (Sumatera); Malaysia (Peninsular Malaysia);,NaN,['\n The species occurs in some forest reserves.\n\n \n '],...,-33.0,2.0,0,0.500000,1.0,0.0,0,0.0,1,0.0
2,7553756,2846,954,"blackfaced black spider monkey, chamek spider monkey, peruvian black spider monkey, peruvian spi...",Ateles,Atelidae,Terrestrial,"Bolivia, Plurinational States of; Brazil (Acre, Amazonas, Mato Grosso, RondÃ?Â´nia); Peru;",\r\r\r\r\n The major threat is subsistence and market hunting for food (with guns). An additi...,"['\n This species is confirmed, or may occur, in numerous protected areas.', <br/>, <br/>, 'B...",...,-87.0,0.0,0,0.000000,1.0,0.0,0,0.0,0,1.0
4,8371955,5800,1057,"black sea bass, giant sea bass",Stereolepis,Polyprionidae,Marine,"<div>Pacific Ã¢?? eastern central</div>; Mexico (Baja California, Sonora); United States (Califo...","\r\r\r\r\n The major threat is from overfishing; however, the species is protected in Califor...",['\n The fish has been protected in California since 1982 and protected in Mexico since 1992 ...,...,-89.0,2.0,0,0.333333,1.0,0.0,1,0.0,0,1.0
5,1780670,7710,224,largefin pupfish,Cyprinodon,Cyprinodontidae,NaN,Mexico;,NaN,NaN,...,-25.0,3.0,0,0.333333,1.0,0.0,0,0.0,0,0.0
7,1840302,4022,232,chamula mountain brook frog,Duellmanohyla,Hylidae,Terrestrial; Freshwater,Mexico;,\r\r\r\r\n The main threat is deforestation and the disturbance of cloud forest remnants (at ...,"['\n The range of this species does not include any protected areas, and the protection and r...",...,-10.0,1.0,0,0.500000,1.0,0.0,1,0.0,1,0.0
9,8990394,6869,1135,venda cycad,Encephalartos,Zamiaceae,Terrestrial,South Africa (Limpopo Province);,\r\r\r\r\n This species is threatened due to over-collecting for ornamental purposes.\r\r\r\r...,['\n This species is listed on Appendix I of the CITES Appendices. Plants occur (or occurred)...,...,-20.0,2.0,1,1.000000,0.0,1.0,1,0.0,1,0.0
10,2107152,1762,266,white meranti,Shorea,Dipterocarpaceae,Terrestrial,Malaysia (Peninsular Malaysia);,\r\r\r\r\n The tree is slow-growing and cannot withstand logging because the reproductive cyc...,NaN,...,3.0,7.0,0,0.333333,1.0,0.0,1,0.0,0,0.0
12,6244442,7422,788,tanners reed frog,Hyperolius,Hyperoliidae,Terrestrial; Freshwater,"Tanzania, United Republic of;",\r\r\r\r\n The forest habitat in the West Usambara Mountains is being lost due to agricultura...,['\n The species occurs in or near protected habitats (Mazumbai and Shume-Magambo Forest Rese...,...,-13.0,2.0,1,1.000000,0.0,1.0,1,0.0,1,0.0
14,5711867,5152,721,attenboroughs pitcher plant,Nepenthes,Nepenthaceae,Terrestrial,Philippines;,\r\r\r\r\n The main threat to this species is from poaching; the mountain is relatively inacc...,"['\n There are no ', <span style=""font-style: italic;"">in situ</span>, "" conservation measure...",...,-21.0,2.0,0,0.250000,1.0,0.0,1,0.0,0,0.0
16,8974565,6870,1133,lydenburg cycad,Encephalartos,Zamiaceae,Terrestrial,South Africa (Limpopo Province);,"\r\r\r\r\n This species has suffered much from the activities of collectors and in addition, ...",['\n This species is listed on Appendix I of the CITES Appendices.\n\n \n '],...,-9.0,12.0,1,1.000000,0.0,1.0,1,1.0,1,0.0


In [ ]:
add = pd.DataFrame(prediction, columns=['label'])
df['label'] = add
df = df[df.label != 0]